In [1]:
CHARACTER = "ФИБИ"

main_characters = ['ДЖОУИ', 'МОНИКА', 'РЕЙЧЕЛ', 'РОСС', 'ФИБИ', 'ЧЕНДЛЕР']
labels_to_chars = dict(enumerate(main_characters))
chars_to_labels = {v: k for k, v in labels_to_chars.items()}
labels_to_chars, chars_to_labels

({0: 'ДЖОУИ', 1: 'МОНИКА', 2: 'РЕЙЧЕЛ', 3: 'РОСС', 4: 'ФИБИ', 5: 'ЧЕНДЛЕР'},
 {'ДЖОУИ': 0, 'МОНИКА': 1, 'РЕЙЧЕЛ': 2, 'РОСС': 3, 'ФИБИ': 4, 'ЧЕНДЛЕР': 5})

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git clone https://github.com/Alenush/style_transfer_sirius2021summer.git

Cloning into 'style_transfer_sirius2021summer'...
remote: Enumerating objects: 1727, done.
remote: Counting objects: 100% (1727/1727), done.
remote: Compressing objects: 100% (1075/1075), done.
remote: Total 1727 (delta 765), reused 1582 (delta 625), pack-reused 0
Receiving objects: 100% (1727/1727), 49.46 MiB | 12.01 MiB/s, done.
Resolving deltas: 100% (765/765), done.


In [4]:
%%bash

cd style_transfer_sirius2021summer
git checkout master
git pull
cd -

Branch 'master' set up to track remote branch 'master' from 'origin'.
Already up to date.
/content


Switched to a new branch 'master'


In [5]:
chars_ru2en = {
    'ДЖОУИ'  : 'Joey',
    'МОНИКА' : 'Monica',
    'РЕЙЧЕЛ' : 'Rachel',
    'РОСС'   : 'Ross',
    'ФИБИ'   : 'Phoebe',
    'ЧЕНДЛЕР': 'Chandler',
}
chars_ru2en

{'ДЖОУИ': 'Joey',
 'МОНИКА': 'Monica',
 'РЕЙЧЕЛ': 'Rachel',
 'РОСС': 'Ross',
 'ФИБИ': 'Phoebe',
 'ЧЕНДЛЕР': 'Chandler'}

In [52]:
#!g1.1
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [53]:
import pathlib

# The name of the task to train.I'm going to name this 'yelp'.
EN_CHAR_NAME = chars_ru2en[CHARACTER].lower()
TASK_NAME = f'en-{EN_CHAR_NAME}'

ROOT_DIR = '/content/drive/MyDrive/FriendsClassifiers'

# The output directory where the fine-tuned model and checkpoints will be written.
OUTPUT_DIR = f'{ROOT_DIR}/outputs/{TASK_NAME}/'

WEIGHTS_NAME = f"{TASK_NAME}-pytorch_model.bin"
NORMAL_WEIGHTS_NAME = "pytorch_model.bin"

In [54]:
OUTPUT_DIR

'/content/drive/MyDrive/FriendsClassifiers/outputs/en-phoebe/'

In [55]:
import os

try:
    os.rename(
        f'{OUTPUT_DIR}{WEIGHTS_NAME}',
        f'{OUTPUT_DIR}{NORMAL_WEIGHTS_NAME}')
except FileNotFoundError:
    print("Already renamed")

Already renamed


In [56]:
!pip install transformers

In [57]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load a trained model and vocabulary that you have fine-tuned
model = BertForSequenceClassification.from_pretrained(OUTPUT_DIR)
tokenizer = BertTokenizer.from_pretrained(OUTPUT_DIR)

# Copy the model to the GPU.
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [21]:
import pandas as pd

df = pd.read_csv("/content/style_transfer_sirius2021summer/data/train_data/en/only_unique_phrases_eng.csv")
df

,label,sentence,base_sentence
0,0,"Hey, you've traveled a lot right?",hey youve travel a lot right
1,1,"But, we pick again!",but we pick again
2,5,"Ok, ok, you can be shirts and I'll be skins.",ok ok you can be shirt and be skin
3,3,"Oh, but he will.",oh but he will
4,2,"Whoa, that Diet Coke just went straight to my ...",whoa that just go straight to my head
...,...,...,...
59291,0,"Ross, Ross, over here man.",over here man
59292,3,"Uhh, it was fun.",uhh it be fun
59293,5,"Okay, I'll give them back.",okay give them back
59294,5,"Well, thats not true, hehe smacked you once.",well thats not true hehe smack you once


In [22]:
binarized_dfs = dict()

for character in chars_to_labels:
    binarized_dfs[character] = df.copy()
    binarized_dfs[character]["label"] = binarized_dfs[character]["label"].apply(
        lambda x: 1 if chars_to_labels[character] == x else 0
    )
    
df_char = binarized_dfs[CHARACTER]
df_char

,label,sentence,base_sentence
0,0,"Hey, you've traveled a lot right?",hey youve travel a lot right
1,0,"But, we pick again!",but we pick again
2,0,"Ok, ok, you can be shirts and I'll be skins.",ok ok you can be shirt and be skin
3,0,"Oh, but he will.",oh but he will
4,0,"Whoa, that Diet Coke just went straight to my ...",whoa that just go straight to my head
...,...,...,...
59291,0,"Ross, Ross, over here man.",over here man
59292,0,"Uhh, it was fun.",uhh it be fun
59293,0,"Okay, I'll give them back.",okay give them back
59294,0,"Well, thats not true, hehe smacked you once.",well thats not true hehe smack you once


In [28]:
f"Negative = {round(100 * df_char[df_char['label'] == 0].shape[0] / df_char.shape[0])}%"

'Negative = 85%'

In [65]:
df_neg_ = df_char[df_char['label'] == 0].sample(60).reset_index(drop=True)
df_neg_

,label,sentence,base_sentence
0,0,"Oh yeah, what's it about?",oh yeah whats it about
1,0,Because she means nothing to me!,because she mean nothing to me
2,0,We're supposed to have sex tonight.,were suppose to have sex tonight
3,0,She was the one who turned down the lights.,she be the one who turn down the light
4,0,"Come on little girl, hi!",come on little girl hi
5,0,"Oh my God, the adoption lady is early!",oh my the adoption lady be early
6,0,Did it turn into sand?,did it turn into sand
7,0,I got it from the gift shop!,i get it from the gift shop
8,0,We can still get out.,we can still get out
9,0,"Mm, no.",mm no


In [66]:
df_pos_ = df_char[df_char['label'] == 1].sample(n=40).reset_index(drop=True)
df_pos_

,label,sentence,base_sentence
0,1,"Joey, you've you've fixed him up with his per...",youve youve fix him up with his perfect woman
1,1,It's a normal Swedish name.,its a normal swedish name
2,1,She is gonna hate me.,she be gon na hate me
3,1,"Well, 'cause then they'd be like my mom used t...",well cause then theyd be like my mom use to ma...
4,1,Its freshwater eel!,its freshwater eel
5,1,But everybody sings.,but everybody sings
6,1,And I dont want to end up like that with Rachel.,and i dont want to end up like that with
7,1,"Uh, wow!",uh wow
8,1,"Well, I have plans with Joey tonight.",well i have plan with tonight
9,1,I just thought we might be here for awhile.,i just thought we might be here for awhile


In [67]:
result = pd.concat([df_pos_, df_neg_])
df_eval = result.sample(n=result.shape[0]).reset_index(drop=True)[["sentence", "label"]]
df_eval

,sentence,label
0,"Uhu, uhu, well, then, great.",1
1,Promise?,1
2,She's giving us her baby.,0
3,"Oh right, ok um.",1
4,"Well, I have plans with Joey tonight.",1
...,...,...
95,Remember that thing that's been dead for a gaz...,0
96,"They, they have a liking problem with you.",1
97,It's your hair.,0
98,If that is your father calling to tell this st...,0


In [68]:
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler

# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(df_eval.shape[0]))

# Create sentence and label lists
sentences = df_eval.sentence.values
labels = df_eval.label.values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Set the batch size.  
batch_size = 32  

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Number of test sentences: 100



/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [69]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 100 test sentences...
    DONE.


In [71]:
import numpy as np

predictions_lst = []
for i in predictions:
    predictions_lst += i.tolist()
predictions_lst= np.array(predictions_lst)
predictions_lst = np.argmax(predictions_lst,  axis=1).flatten()

true_labels_lst = []
for i in true_labels:
    true_labels_lst += i.tolist()
true_labels_lst= np.array(true_labels_lst)

In [72]:
(predictions_lst == true_labels_lst).sum() / true_labels_lst.shape[0]

0.6

In [77]:
from sklearn.metrics import matthews_corrcoef
from scipy.stats import pearsonr

In [75]:
matthews_corrcoef(predictions_lst, true_labels_lst)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


0.0

In [79]:
pearsonr(predictions_lst, true_labels_lst)

/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


(nan, nan)

In [81]:
from scipy.stats import ttest_ind
ttest_ind(predictions_lst, true_labels_lst)

Ttest_indResult(statistic=-8.12403840463596, pvalue=4.7738117806692215e-14)